In [1]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import pandas as pd

In [2]:
# Define the base class for declarative models
Base = declarative_base()

C:\Users\User\AppData\Local\Temp\ipykernel_9260\2123928503.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
# Define the School Data model
class SchoolData(Base):
    __tablename__ = 'SchoolData'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    lea_name = Column(String)
    school_entity_id = Column(Integer)
    school_name = Column(String)
    county = Column(String)
    subgroup = Column(String)
    number_graduated = Column(Integer)
    number_in_cohort = Column(Integer)
    percent_graduated = Column(Float)

In [4]:
# Database setup
engine = create_engine('sqlite:///merged_cohorts_df.db')  # Use your existing DB file
Base.metadata.create_all(engine)  # Create tables if they don't exist
Session = sessionmaker(bind=engine)

In [5]:
# Load data from CSV
data = pd.read_csv('merged_cohorts_df.csv')

In [8]:
print("Columns in the DataFrame:", data.columns)

Columns in the DataFrame: Index(['LEA Name', 'School Entity ID', 'School Name', 'County', 'Subgroup',
       'Number Graduated', 'Number in Cohort', 'Percent Graduated in 4 Years'],
      dtype='object')


In [9]:
# Insert data into the database
with Session() as session:
    for _, row in data.iterrows():
        # Handle '*' by converting to None
        number_graduated = row['Number Graduated'] if row['Number Graduated'] != '*' else None
        number_in_cohort = row['Number in Cohort'] if row['Number in Cohort'] != '*' else None
        
        # Clean and convert percent graduated
        percent_graduated_str = row['Percent Graduated in 4 Years']
        if percent_graduated_str == '*' or not percent_graduated_str:
            percent_graduated = None
        else:
            # Handle various cases for percent graduated
            if '<' in percent_graduated_str:  # Example: '< 2%'
                percent_graduated = 0.0  # Set to 0.0 or another value if needed
            elif '>' in percent_graduated_str:  # Example: '> 98'
                percent_graduated = 100.0  # Set to 100.0 for percentages above 98
            else:
                try:
                    # Remove the '%' sign and convert to float
                    percent_graduated = float(percent_graduated_str.replace('%', '').strip())
                except ValueError:
                    percent_graduated = None  # Handle cases that still fail

        # Create entry
        entry = SchoolData(
            lea_name=row['LEA Name'],
            school_entity_id=row['School Entity ID'],
            school_name=row['School Name'],
            county=row['County'],
            subgroup=row['Subgroup'],
            number_graduated=number_graduated,
            number_in_cohort=number_in_cohort,
            percent_graduated=percent_graduated
        )
        session.merge(entry)  # Use merge to avoid duplicates

    session.commit()


In [10]:
# Read data from the database and display it
with Session() as session:
    result = session.query(SchoolData).all()
    df = pd.DataFrame([(d.lea_name, d.school_entity_id, d.school_name, d.county,
                        d.subgroup, d.number_graduated, d.number_in_cohort, d.percent_graduated) for d in result],
                      columns=['LEA Name', 'School Entity ID', 'School Name', 'County', 
                               'Subgroup', 'Number Graduated', 'Number in Cohort', 'Percent Graduated'])
    print(df)

                                  LEA Name  School Entity ID  \
0                  Chinle Unified District              4737   
1                  Chinle Unified District              4737   
2                  Chinle Unified District              4737   
3                  Chinle Unified District              4737   
4                  Chinle Unified District              4737   
...                                    ...               ...   
39203  Arizona Goodwill Education Services           1001974   
39204  Arizona Goodwill Education Services           1001974   
39205  Arizona Goodwill Education Services           1001974   
39206  Arizona Goodwill Education Services           1001974   
39207  Arizona Goodwill Education Services           1001974   

                      School Name    County            Subgroup  \
0              Chinle High School    Apache          ELL Cohort   
1              Chinle High School    Apache                 All   
2              Chinle High Sch